# Module 1 - Calculating seasonal maps and resampling WaPOR climate data (RET & PCP)
This Notebook contains the following steps
* Step a - Import modules/libraries
* Step b - Calculate seasonal PCP and RET data
* Step c - Resampling seasonal PCP and RET raster data

**=====================================================================================================================**

![image](https://raw.githubusercontent.com/wateraccounting/WaPORIPA/main/images/Resampling.png)

**=====================================================================================================================**

Before you start running this Notebook, make sure you have the following information:

*   WaPOR Reference ET and PCP for your case study for the period you want to do the analyses. Use this script for downloading the data: [Downloading_WaPORv3_data script](https://github.com/wateraccounting/WaPORMOOC/tree/main/1_WaPOR_download_colab). *For small areas make sure you download the data using a bounding box to avoid non data*

*   Sample of a TIFF file with the resolution you want to do the analyses

## Step a - Import modules/libraries

In [ ]:
!pip install --upgrade rioxarray --quiet

In [2]:
import os                             # a module for interacting with the operating system
import glob                           # used to retrieve files/pathnames matching a specified pattern
import matplotlib.pyplot as plt       # is a plotting library used for 2D graphics in python
import numpy as np                    # stands for 'Numerical Python, is a python library used for scientific computing with arrays
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rioxr
import rasterio as rio
import pandas as pd                       # to store and manipulate tabular data in rows of observations and columns of variables
import calendar
import datetime

from osgeo import ogr, gdal
import subprocess
from rasterio.enums import Resampling
from rasterio.warp import reproject

###Import downloaded RET and PCP files

Import WaPOR RET and PCP obtained using [Downloading_WaPORv3_data script](https://github.com/wateraccounting/WaPORMOOC/tree/main/1_WaPOR_download_colab).

NOTE: *Download using bounding box (bb) not shapefile to avoid non data.*

In [ ]:
#To upload data.zip.
from google.colab import files
uploaded = files.upload()

In [ ]:
# unzip the uploaded zipfile
!unzip '/content/data.zip' -d '/content/'

# Step b - Create seasonal climate layers


###Create input and output folders

In [ ]:
target_folderPCP = '/content/content/output/L1-PCP-D'                       # location of data to be aggregated
target_fhsPCP    = glob.glob(target_folderPCP + '/*.tif')
output_folderPCP_season = "/content/content/seasonal/L1-PCP-season"

if not os.path.exists(output_folderPCP_season):
    os.makedirs(output_folderPCP_season)

target_folderRET = '/content/content/output/L1-RET-D'                      # location of data to be aggregated
target_fhsRET    = glob.glob(target_folderRET + '/*.tif')
output_folderRET_season = "/content/content/seasonal/L1-RET-season"

if not os.path.exists(output_folderRET_season):
    os.makedirs(output_folderRET_season)

output_folderPCP_season, output_folderRET_season

###Define seasons

In [25]:
season_periods = {
    'season1': {'SOS': '2022-10-01', 'EOS': '2023-04-30'}
}

###Define function for temporal aggregation

This script defines the function `SumSeason `which is used in the next sections. We need this function to aggregate PCP and RET data.


In [26]:
# summation of raster between two dates
def SumSeason(input_fhs, sowing_date, harvesting_date):

    # add rasters that falls within sowing and harvesting date
    Sums = 0
    for i, in_fh in enumerate(input_fhs):
        # open raster file and get start and end date from attributes
        ds = rioxr.open_rasterio(in_fh)
        ds = ds.where(ds!=ds.attrs['_FillValue'])
        if i == 0:
          attrs = ds.attrs
        raster_startdate = datetime.datetime.strptime(ds.attrs['start_date'], '%Y-%m-%d' ).date()
        raster_enddate   = datetime.datetime.strptime(ds.attrs['end_date'], '%Y-%m-%d' ).date()

        # accumulate full or part of the dekadal value
        if (raster_startdate >= sowing_date) & (raster_enddate <= harvesting_date):
            Sum = ds
            Sums += Sum
#             print ("1",in_fh)

        elif (raster_startdate <sowing_date)&(raster_enddate >sowing_date)&(raster_enddate <harvesting_date):
            Sum = ds
            Sum = Sum*((raster_enddate-sowing_date)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("2",in_fh)

        elif (raster_startdate >sowing_date)&(raster_startdate <harvesting_date)&(raster_enddate >harvesting_date):
            Sum = ds
            Sum = Sum*((harvesting_date- raster_startdate)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("3",in_fh)

        elif (sowing_date>=raster_startdate)&(harvesting_date<=raster_enddate):
            Sum = ds
            Sum = Sum*((harvesting_date- sowing_date)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("4",in_fh)
    attrs.update({'start_date':datetime.datetime.strftime(SOS, '%Y-%m-%d'),
              'end_date': datetime.datetime.strftime(EOS, '%Y-%m-%d'),
              'units' : 'mm/season'})
    del attrs['number_of_days']
    del attrs['temporal_resolution']
    del attrs['units_conversion_factor']

    Sums.attrs  = attrs
    return Sums

##Calculate seasonal WaPOR RET and PCP
### i) Calculate seasonal Reference ET

In [28]:
# Assign inputs to the SumSeason function
input_fhs      = target_fhsRET
output_folder  = output_folderRET_season

for i, season in enumerate(season_periods):
    # calculate the seasonal value and save in output_folder
    SOS = datetime.datetime.strptime(season_periods[season]['SOS'],'%Y-%m-%d').date()
    EOS = datetime.datetime.strptime(season_periods[season]['EOS'],'%Y-%m-%d').date()
    seasonal = SumSeason(input_fhs, SOS, EOS)

    ## save the array in raster format, name it with the raster_id and sowing and harvesting date
    out_fh = os.path.join(output_folder, 'RET' + '_' + season + '_' + season_periods[season]['SOS']+'_to_'+season_periods[season]['EOS']+ '.tif')
    seasonal.rio.to_raster(out_fh)

### ii) Calculate seasonal precipitation

In [27]:
# Assign inputs to the SumSeason function
input_fhs = target_fhsPCP
output_folder  = output_folderPCP_season


for i, season in enumerate(season_periods):
    # calculate the seasonal value and save in output_folder
    SOS = datetime.datetime.strptime(season_periods[season]['SOS'],'%Y-%m-%d').date()
    EOS = datetime.datetime.strptime(season_periods[season]['EOS'],'%Y-%m-%d').date()
    seasonal = SumSeason(input_fhs, SOS, EOS)

    ## save the array in raster format, name it with the raster_id and sowing and harvesting date
    out_fh = os.path.join(output_folder, 'PCP'+ '_' + season + '_' + season_periods[season]['SOS']+'_to_'+season_periods[season]['EOS']+ '.tif')
    seasonal.rio.to_raster(out_fh)


# Step c - Resample seasonal raster data



The data needs to be resampled to the same resolution as the NPP and AETI data to be able to use in the next module. You need an example file (TIFF file) with this resolution at hand.

Import the example file with the required resolution (WaPORv3 or pyWaPOR output file).

In [ ]:
#To upload sample resolution file.
from google.colab import files
uploaded = files.upload()

Create input and output folders

In [ ]:
source_file   = '/content/pywapor_AETI_D_2022-10-01.tif'        # Read gdal info of template raster file
target_folder = '/content/content/seasonal/L1-RET-season'                           # folder of RET data to be resampled
target_fhsRET_season =  glob.glob(target_folder + '/*.tif')
target_folder = '/content/content/seasonal/L1-PCP-season'                      # folder of PCP data to be resampled
target_fhsPCP_season    = glob.glob(target_folder + '/*.tif')

# create output folder
output_folderRET = '/content/content/pywapor_resampled/L1-RET-season_resampled'
output_folderPCP = '/content/content/pywapor_resampled/L1-PCP-season_resampled'

## Make one if the folder does not exit
if not os.path.exists(output_folderRET):
    os.makedirs(output_folderRET)

if not os.path.exists(output_folderPCP):
    os.makedirs(output_folderPCP)

source_file, target_folder, target_fhsPCP_season, output_folderPCP,
source_file, target_folder, target_fhsRET_season, output_folderRET

Define function to resample raster data

In [51]:
#Function to resample raster
def resample_raster(source_file, target_files, output_folder, resample_method=Resampling.nearest, dtype='float32'):
    with rio.open(source_file) as src:
        source_profile = src.profile
        source_transform = src.transform
        source_crs = src.crs

    for target_file in target_files:
        with rio.open(target_file) as tgt:
            target_data = tgt.read(1)
            resampled_data = np.empty(shape=(source_profile['height'], source_profile['width']), dtype=dtype)

            reproject(
                source=target_data,
                destination=resampled_data,
                src_transform=tgt.transform,
                src_crs=tgt.crs,
                dst_transform=source_transform,
                dst_crs=source_crs,
                resampling=resample_method
            )

            # Update profile for the resampled file
            resampled_profile = source_profile.copy()
            resampled_profile.update(dtype=dtype)

            # Define the output file path
            basename = os.path.basename(target_file)
            output_file = os.path.join(output_folder, basename)

            with rio.open(output_file, 'w', **resampled_profile) as dst:
                dst.write(resampled_data, 1)

            # Print the size and shape of the resampled raster file
            print(f'Resampled {basename}: size = {resampled_data.size}, shape = {resampled_data.shape}')


### i) Resample Reference ET

Check the size and shape of the raster files

In [ ]:
with rio.open(source_file) as src:
    print('The size & shape of the template raster      =', src.width * src.height, '&', (src.height, src.width))

with rio.open(target_fhsRET_season[0]) as tgt:
    print('The size & shape of the data to be resampled =', tgt.width * tgt.height, '&', (tgt.height, tgt.width))

Resample raster data

In [ ]:
# Call the function
resample_raster(source_file, target_fhsRET_season, output_folderRET)

### ii) Resample Precipitation

Check the size and shape of the raster files

In [ ]:
with rio.open(source_file) as src:
    print('The size & shape of the template raster      =', src.width * src.height, '&', (src.height, src.width))

with rio.open(target_fhsPCP_season[0]) as tgt:
    print('The size & shape of the data to be resampled =', tgt.width * tgt.height, '&', (tgt.height, tgt.width))



Resample raster data

In [ ]:
# Call the function
resample_raster(source_file, target_fhsPCP_season, output_folderPCP)

##Zip and download resampled data

In [ ]:
!zip -r /content/pywapor_resampled.zip /content/content/pywapor_resampled/
from google.colab import files
files.download(r'/content/pywapor_resampled.zip')


In [34]:
# # if you want to delete a folder use the code below.
# !rm -rf /content/content/seasonal_resampled   # remove folder
# !rm -rf /content/tifs                         # remove tiffs inside folder
# !rm -rf /content/data.zip                     # remove file